In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Read in the locations of the parts that are labeled on each image, and join them with the address of the corresponding images. This is to be able to train a classifier to distinguish between each `part_id` by looking at that part in the image.

In [ ]:
# read in part locations
part_locs = pd.read_csv('CUB_200_2011/CUB_200_2011/parts/part_locs.txt', sep=' ', header=None)
part_locs.columns = ['image_id', 'part_id', 'x', 'y', 'visible']

# read in image ids and their addresses
im_id_addrs = pd.read_csv('CUB_200_2011/CUB_200_2011/images.txt', sep=' ', header=None)
im_id_addrs.columns = ['image_id', 'image_addr']

im_addr_parts = im_id_addrs
# join part locations and image id dataframes in order to get (part, image address) mappings
# im_addr_parts = pd.merge(part_locs, im_id_addrs, on='image_id')

# read in bounding boxes per image id
im_bbs = pd.read_csv('CUB_200_2011/CUB_200_2011/bounding_boxes.txt', sep=' ', header=None)
im_bbs.columns = ['image_id', 'bb_x', 'bb_y', 'bb_w', 'bb_h']

im_addr_parts = pd.merge(im_addr_parts, im_bbs, on='image_id')

im_classes = pd.read_csv('CUB_200_2011/CUB_200_2011/image_class_labels.txt', sep=' ', header=None)
im_classes.columns = ['image_id', 'class_id']

im_addr_parts = pd.merge(im_addr_parts, im_classes, on='image_id')

# read in part labels manually because parts can be two words
# part_labels = []
# with open('CUB_200_2011/CUB_200_2011/parts/parts.txt') as part_labels_file:
#     for row in part_labels_file:
#         sep = row.find(' ')
#         part_id, part_name = row[:sep], row[sep + 1:-1]
#         part_labels.append({'part_id': int(part_id), 'part_name': part_name})
# part_labels = pd.DataFrame.from_dict(part_labels)

# join (part, image addresses) with part labels to make part_id's comprehensible
# im_addr_parts = pd.merge(im_addr_parts, part_labels, on='part_id')
print(im_addr_parts.head(30))

In [1]:
NUM_CLASSES = 20 # limit classes trained

import matplotlib.patches as patches
from collections import defaultdict
from skimage import io, transform

# GENERATE THE TRAINING DATA

# CLASS_IMGS = defaultdict(list)
# first level key = class_id. second level key = part_id

class_ids = set()

CLASS_INSTANCES = []
IMG_LABELS = []
# prev_class_id = 0

im_addr_classes = pickle.load(open('metadata', 'rb'))
for index, row in im_addr_classes.iterrows():
    im_id, im_addr, bb_x, bb_y, bb_w, bb_h, class_id = row
    
    if class_id not in class_ids:
        if len(class_ids) >= NUM_CLASSES:
            break
        class_ids.add(class_id)
    
    img = io.imread('CUB_200_2011/images/' + im_addr)
    
    bb_x, bb_y, bb_w, bb_h = [int(x) for x in [bb_x, bb_y, bb_w, bb_h]]
    if len(img.shape) == 2:
        img = np.dstack([img, img, img])
    if 0 in img.shape:
        continue
    plt.imshow(img)
    plt.show()
    break
        
    img = img[bb_y:bb_y + bb_h, bb_x:bb_x + bb_w, :]
    img = transform.resize(img, (224, 224, 3))


#     print('extracting foreground for', class_id, im_addr)
    
    CLASS_INSTANCES.append(img)
    IMG_LABELS.append(class_id)

print(np.array(CLASS_INSTANCES).shape)

NameError: name 'pickle' is not defined

In [ ]:
# DON"T RUN THIS

all_imgs_processed = []
all_labels = []
for class_id, filename in enumerate(filenames):
    class_imgs = pickle.load(open(filename, 'rb'))
    for img_idx, img in enumerate(class_imgs):
#         crops = []
#         actual_num_crops = 0
#         for i in range(NUM_CROPS):
#             rand_x = randrange(0, img.shape[0] - CROP_SIZE + 1)
#             rand_y = randrange(0, img.shape[1] - CROP_SIZE + 1)
#             crop = img[rand_y:rand_y + CROP_SIZE, rand_x:rand_x + CROP_SIZE]
#             if crop.shape != (100, 100, 3):
#                 continue
#             crops.append(crop)
#             actual_num_crops += 1
#         all_imgs_processed += crops
#         all_labels += [class_id] * actual_num_crops
        x_center = img.shape[0] // 2
        y_center = img.shape[1] // 2
        crop = img[y_center - CROP_SIZE//2:y_center + CROP_SIZE//2, x_center - CROP_SIZE//2:x_center + CROP_SIZE//2]
        if crop.shape != (100, 100, 3):
            continue
        all_imgs_processed += [crop]
        all_labels += [class_id]

In [3]:
all_imgs_processed = np.array(CLASS_INSTANCES)
all_labels = np.array(IMG_LABELS)
# np.savez_compressed('images_normalized', all_imgs_processed)
# np.savez_compressed('images_normalize_labels', all_labels)

In [4]:
# VGGNet training

# all_imgs_processed = np.load('images_normalized.npz')
# all_labels = np.load('images_normalized_labels.npz')
import vgg as vgg_mod
import importlib
importlib.reload(vgg_mod)

sess = tf.Session()
imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg_obj = vgg_mod.vgg16(imgs, 'vgg16_weights.npz', sess)

labels_ph = tf.placeholder(tf.int64, (None), name='labels_ph')
labels_ph_oh = tf.one_hot(labels_ph, 20)
print(labels_ph_oh)

loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=vgg_obj.logits, labels=labels_ph_oh)
optimizer = tf.train.AdamOptimizer(0.0001, 0.9, 0.999)
opt = optimizer.minimize(loss)

correct = tf.equal(tf.argmax(vgg_obj.probs, 1), labels_ph)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
# # Unused prediction code

# prob = sess.run(vgg.probs, feed_dict={vgg.imgs: [img1]})[0]

# preds = (np.argsort(prob)[::-1])[0:5]
# for p in preds:
#     print class_names[p], prob[p]

Tensor("one_hot:0", dtype=float32)


In [5]:
# init = tf.initialize_variables([vgg_obj.fc1, vgg_obj.fc2, vgg_obj.fc31])
# print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'fc31'))
sess.run(tf.global_variables_initializer())
vgg_obj.load_weights(vgg_obj.weights, vgg_obj.sess)
# for varname in ['fc1', 'fc2', 'fc31']:
#     coll = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'fc31')
#     for var in coll:
#         sess.run(var.initializer)
# # sess.run(init)
# sess.run(sess.graph.get_tensor_by_name('beta1_power/Assign:0'))
# sess.run(sess.graph.get_tensor_by_name('beta2_power/Assign:0'))

0 conv1_1_W (3, 3, 3, 64)
1 conv1_1_b (64,)
2 conv1_2_W (3, 3, 64, 64)
3 conv1_2_b (64,)
4 conv2_1_W (3, 3, 64, 128)
5 conv2_1_b (128,)
6 conv2_2_W (3, 3, 128, 128)
7 conv2_2_b (128,)
8 conv3_1_W (3, 3, 128, 256)
9 conv3_1_b (256,)
10 conv3_2_W (3, 3, 256, 256)
11 conv3_2_b (256,)
12 conv3_3_W (3, 3, 256, 256)
13 conv3_3_b (256,)
14 conv4_1_W (3, 3, 256, 512)
15 conv4_1_b (512,)
16 conv4_2_W (3, 3, 512, 512)
17 conv4_2_b (512,)
18 conv4_3_W (3, 3, 512, 512)
19 conv4_3_b (512,)
20 conv5_1_W (3, 3, 512, 512)
21 conv5_1_b (512,)
22 conv5_2_W (3, 3, 512, 512)
23 conv5_2_b (512,)
24 conv5_3_W (3, 3, 512, 512)
25 conv5_3_b (512,)
26 fc6_W (25088, 4096)


In [6]:
# Training code
BS = 8

image_data = all_imgs_processed
label_data = all_labels
print(image_data.shape, label_data.shape)

training = vgg_obj.training
keep_prob = vgg_obj.keep_prob

for epoch in range(50):
    # Let's shuffle the data every epoch
    np.random.seed(epoch)
    np.random.shuffle(image_data)
    np.random.seed(epoch)
    np.random.shuffle(label_data)
    # Go through the entire dataset once
    accuracy_vals, loss_vals = [], []
    for i in range(0, image_data.shape[0]-BS+1, BS):
        # Train a single batch
        batch_images, batch_labels = image_data[i:i+BS], label_data[i:i+BS]
        accuracy_val, loss_val, _ = sess.run([accuracy, loss, opt], feed_dict={imgs: batch_images, labels_ph: batch_labels, training: True, keep_prob: 0.9})
        accuracy_vals.append(accuracy_val)
        loss_vals.append(loss_val)

#     val_correct = []
#     for i in range(0, image_val.shape[0], BS):
#         batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
#         val_correct.extend( sess.run(correct, feed_dict={eval_inputs: batch_images, labels_ph: batch_labels}) )
    print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f '%(epoch, np.mean(accuracy_vals), np.mean(loss_vals)))

(1115, 224, 224, 3) (1115,)
[  0] Accuracy: 0.065  	  Loss: 0.807 
[  1] Accuracy: 0.121  	  Loss: 0.790 
[  2] Accuracy: 0.133  	  Loss: 0.781 
[  3] Accuracy: 0.139  	  Loss: 0.781 
[  4] Accuracy: 0.136  	  Loss: 0.780 
[  5] Accuracy: 0.149  	  Loss: 0.775 
[  6] Accuracy: 0.128  	  Loss: 0.777 
[  7] Accuracy: 0.138  	  Loss: 0.778 
[  8] Accuracy: 0.115  	  Loss: 0.784 
[  9] Accuracy: 0.154  	  Loss: 0.778 
[ 10] Accuracy: 0.142  	  Loss: 0.774 
[ 11] Accuracy: 0.160  	  Loss: 0.776 
[ 12] Accuracy: 0.167  	  Loss: 0.770 
[ 13] Accuracy: 0.181  	  Loss: 0.769 
[ 14] Accuracy: 0.164  	  Loss: 0.770 
[ 15] Accuracy: 0.171  	  Loss: 0.768 
[ 16] Accuracy: 0.165  	  Loss: 0.768 
[ 17] Accuracy: 0.188  	  Loss: 0.760 
[ 18] Accuracy: 0.183  	  Loss: 0.767 
[ 19] Accuracy: 0.184  	  Loss: 0.764 
[ 20] Accuracy: 0.185  	  Loss: 0.764 
[ 21] Accuracy: 0.201  	  Loss: 0.765 
[ 22] Accuracy: 0.196  	  Loss: 0.766 
[ 23] Accuracy: 0.189  	  Loss: 0.766 
[ 24] Accuracy: 0.183  	  Loss: 0.76

In [ ]:
################################################################################
#Michael Guerzhoy and Davi Frossard, 2016
#AlexNet implementation in TensorFlow, with weights
#Details: 
#http://www.cs.toronto.edu/~guerzhoy/tf_alexnet/
#
#With code from https://github.com/ethereon/caffe-tensorflow
#Model from  https://github.com/BVLC/caffe/tree/master/models/bvlc_alexnet
#Weights from Caffe converted using https://github.com/ethereon/caffe-tensorflow
#
#
################################################################################

from numpy import *
import os
#from pylab import *
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.cbook as cbook
import time
from scipy.misc import imread
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from numpy import random


import tensorflow as tf

from caffe_classes import class_names

train_x = zeros((1, 100,100,3)).astype(float32)
train_y = zeros((1, 15))
xdim = train_x.shape[1:]
ydim = train_y.shape[1]



################################################################################
#Read Image, and change to BGR


im1 = (imread("laska.png")[:,:,:3]).astype(float32)
im1 = im1 - mean(im1)
im1[:, :, 0], im1[:, :, 2] = im1[:, :, 2], im1[:, :, 0]

im2 = (imread("poodle.png")[:,:,:3]).astype(float32)
im2[:, :, 0], im2[:, :, 2] = im2[:, :, 2], im2[:, :, 0]


################################################################################

# (self.feed('data')
#         .conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
#         .lrn(2, 2e-05, 0.75, name='norm1')
#         .max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
#         .conv(5, 5, 256, 1, 1, group=2, name='conv2')
#         .lrn(2, 2e-05, 0.75, name='norm2')
#         .max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
#         .conv(3, 3, 384, 1, 1, name='conv3')
#         .conv(3, 3, 384, 1, 1, group=2, name='conv4')
#         .conv(3, 3, 256, 1, 1, group=2, name='conv5')
#         .fc(4096, name='fc6')
#         .fc(4096, name='fc7')
#         .fc(1000, relu=False, name='fc8')
#         .softmax(name='prob'))

#In Python 3.5, change this to:
net_data = load(open("bvlc_alexnet.npy", "rb"), encoding="latin1").item()
#net_data = load("bvlc_alexnet.npy").item()

def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    assert c_i%group==0
    assert c_o%group==0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    
    
    if group==1:
        conv = convolve(input, kernel)
    else:
        input_groups =  tf.split(input, group, 3)   #tf.split(3, group, input)
        kernel_groups = tf.split(kernel, group, 3)  #tf.split(3, group, kernel) 
        output_groups = [convolve(i, k) for i,k in zip(input_groups, kernel_groups)]
        conv = tf.concat(output_groups, 3)          #tf.concat(3, output_groups)
    return  tf.reshape(tf.nn.bias_add(conv, biases), [-1]+conv.get_shape().as_list()[1:])



x = tf.placeholder(tf.float32, (None,) + xdim)
print(x)


#conv1
#conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
k_h = 11; k_w = 11; c_o = 96; s_h = 4; s_w = 4
conv1W = tf.Variable(net_data["conv1"][0])
conv1b = tf.Variable(net_data["conv1"][1])
conv1_in = conv(x, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
conv1 = tf.nn.relu(conv1_in)

#lrn1
#lrn(2, 2e-05, 0.75, name='norm1')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn1 = tf.nn.local_response_normalization(conv1,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool1
#max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)


#conv2
#conv(5, 5, 256, 1, 1, group=2, name='conv2')
k_h = 5; k_w = 5; c_o = 256; s_h = 1; s_w = 1; group = 2
conv2W = tf.Variable(net_data["conv2"][0])
conv2b = tf.Variable(net_data["conv2"][1])
conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv2 = tf.nn.relu(conv2_in)


#lrn2
#lrn(2, 2e-05, 0.75, name='norm2')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn2 = tf.nn.local_response_normalization(conv2,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool2
#max_pool(3, 3, 2, 2, padding='VALID', name='pool2')                                                  
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#conv3
#conv(3, 3, 384, 1, 1, name='conv3')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 1
conv3W = tf.Variable(net_data["conv3"][0])
conv3b = tf.Variable(net_data["conv3"][1])
conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv3 = tf.nn.relu(conv3_in)

#conv4
#conv(3, 3, 384, 1, 1, group=2, name='conv4')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 2
conv4W = tf.Variable(net_data["conv4"][0])
conv4b = tf.Variable(net_data["conv4"][1])
conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv4 = tf.nn.relu(conv4_in)


#conv5
#conv(3, 3, 256, 1, 1, group=2, name='conv5')
k_h = 3; k_w = 3; c_o = 256; s_h = 1; s_w = 1; group = 2
conv5W = tf.Variable(net_data["conv5"][0])
conv5b = tf.Variable(net_data["conv5"][1])
conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv5 = tf.nn.relu(conv5_in)

#maxpool5
#max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

alexnet_flattened = tf.contrib.layers.flatten(maxpool5)

fc1 = tf.contrib.layers.fully_connected(alexnet_flattened, 4096, activation_fn=tf.nn.relu)
fc2 = tf.contrib.layers.fully_connected(fc1, 4096, activation_fn=tf.nn.relu)
fc3 = tf.contrib.layers.fully_connected(fc2, 15, activation_fn=None)

fc8 = fc3

################################

# #fc6
# #fc(4096, name='fc6')
# fc6W = tf.Variable(net_data["fc6"][0])
# fc6b = tf.Variable(net_data["fc6"][1])
# fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [-1, int(prod(maxpool5.get_shape()[1:]))]), fc6W, fc6b)

# #fc7
# #fc(4096, name='fc7')
# fc7W = tf.Variable(net_data["fc7"][0])
# fc7b = tf.Variable(net_data["fc7"][1])
# fc7 = tf.nn.relu_layer(fc6, fc7W, fc7b)

# #fc8
# #fc(1000, relu=False, name='fc8')
# fc8W = tf.Variable(net_data["fc8"][0])
# fc8b = tf.Variable(net_data["fc8"][1])
# fc8 = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

#prob
#softmax(name='prob'))
# output = tf.nn.softmax(fc8)
output = tf.identity(fc8)

In [ ]:
# Setting up variables for training

labels_ph = tf.placeholder(tf.int64, (None), name='labels_ph')
labels_ph_oh = tf.one_hot(labels_ph, 15)
print(labels_ph_oh)

loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=labels_ph_oh)
optimizer = tf.train.AdamOptimizer(0.001, 0.9, 0.999)
opt = optimizer.minimize(loss)

print(labels_ph_oh)
correct = tf.equal(tf.argmax(output, 1), tf.argmax(labels_ph_oh, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
# Training code

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

BS = 32

image_data = all_imgs_processed
label_data = all_labels
print(image_data.shape, label_data.shape)

for epoch in range(50):
    # Let's shuffle the data every epoch
    np.random.seed(epoch)
    np.random.shuffle(image_data)
    np.random.seed(epoch)
    np.random.shuffle(label_data)
    # Go through the entire dataset once
    accuracy_vals, loss_vals = [], []
    for i in range(0, image_data.shape[0]-BS+1, BS):
        # Train a single batch
        batch_images, batch_labels = image_data[i:i+BS], label_data[i:i+BS]
        accuracy_val, loss_val, _ = sess.run([accuracy, loss, opt], feed_dict={x: batch_images, labels_ph: batch_labels})
        accuracy_vals.append(accuracy_val)
        loss_vals.append(loss_val)

#     val_correct = []
#     for i in range(0, image_val.shape[0], BS):
#         batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
#         val_correct.extend( sess.run(correct, feed_dict={eval_inputs: batch_images, labels_ph: batch_labels}) )
    print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f '%(epoch, np.mean(accuracy_vals), np.mean(loss_vals)))

In [ ]:
# Original AlexNet code

#Output:


# t = time.time()
# output = sess.run(prob, feed_dict = {x:[im1,im2]})

# for input_im_ind in range(output.shape[0]):
#     inds = argsort(output)[input_im_ind,:]
#     print("Image", input_im_ind)
#     for i in range(5):
#         print(class_names[inds[-1-i]], output[input_im_ind, inds[-1-i]])

# print(time.time()-t)

In [ ]:
import tensorflow as tf

inputs_ph = tf.placeholder(tf.float32, (None, 64, 64, 3), name='inputs')

labels_ph = tf.placeholder(tf.int64, (None), name='labels')

num_classes = 15

filters = tf.contrib.layers.conv2d(inputs=inputs_ph, num_outputs=40, kernel_size=[5, 5], stride=2, padding='same', activation_fn=tf.nn.relu)
h = tf.contrib.layers.conv2d(inputs=filters, num_outputs=70, kernel_size=[4, 4], stride=2, padding='same', activation_fn=tf.nn.relu)
h = tf.contrib.layers.conv2d(inputs=h, num_outputs=120, kernel_size=[4, 4], stride=2, padding='same', activation_fn=tf.nn.relu)
h = tf.contrib.layers.conv2d(inputs=h, num_outputs=170, kernel_size=[4, 4], stride=2, padding='same', activation_fn=tf.nn.relu)
h = tf.contrib.layers.conv2d(inputs=h, num_outputs=num_classes + 1, kernel_size=[4, 4], stride=2, padding='same', activation_fn=None)

h = tf.contrib.layers.flatten(h)

print(h)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=labels_ph))
optimizer = tf.train.AdamOptimizer(0.0005, 0.9, 0.999)
opt = optimizer.minimize(loss)
# correct = tf.equal(tf.argmax())

In [ ]:
BS = 80

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_imgs_ary = np.expand_dims(imgs_array, axis=3).astype(np.float32)
print(input_imgs_ary.dtype)

labels_array = labels_array.astype(np.int64)
for epoch in range(30):
    np.random.seed(epoch)
    np.random.shuffle(input_imgs_ary)
    np.random.seed(epoch)
    np.random.shuffle(labels_array)
    loss_vals = []
    for i in range(0, input_imgs_ary.shape[0]-BS+1, BS):
        # Train a single batch
        batch_images, batch_labels = input_imgs_ary[i:i+BS], labels_array[i:i+BS]
        loss_val, _ = sess.run([loss, opt], feed_dict={inputs_ph: batch_images, labels_ph: batch_labels})
#         accuracy_vals.append(accuracy_val)
        loss_vals.append(loss_val)
    print(np.mean(loss_vals))
    
    # no validation accuracy yet.
#     print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f' %(epoch, np.mean(accuracy_vals)))

Read in the attribute labels on each image.

In [ ]:
from skimage.feature import hog
from skimage import data, exposure

In [ ]:
# read images as input. Filter only for images from 10 classes.
# HOG detector on entire image
# find points from part centers
# Bounding box around part centers (size tbd, energy maximization) to create filters
# inspiration from DPM paper on size of bounding box.
# train each label to be average of bounding box.

# train on 90% of images, holdout on 10%.

In [ ]:
def extract_foreground(camera, do_show = True, extract_bb = False):
    # https://docs.opencv.org/trunk/d8/d83/tutorial_py_grabcut.html

    w, h = camera.shape[:2]
    mask = np.zeros([w, h], np.uint8)
    bgdModel = np.zeros((1,65), np.float64)
    fgdModel = np.zeros((1,65), np.float64)
    rect = (1, 1, w, h)
    cv2.grabCut(camera,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    segmented = camera*mask2[:,:,np.newaxis]
    
    # Return only the bounding box of the foreground
    
    if not extract_bb:
        return segmented
    else:
        # Get contours (edge image) of the foreground-separated image
        # https://docs.opencv.org/3.3.1/dd/d49/tutorial_py_contour_features.html
        segmented_edges = cv2.Canny(segmented, 0, 255)
        _,thresh = cv2.threshold(segmented_edges,127,255,0)
        _,contours,_ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        # find bounding box around the bird (union of all contours points)
        min_x, min_y = [9999999999] * 2
        max_x, max_y = [0] * 2
        for contour in contours:
            for coord in contour:
                coord = coord[0]
                min_x = min(min_x, coord[0])
                max_x = max(max_x, coord[0])
                min_y = min(min_y, coord[1])
                max_y = max(max_y, coord[1])

        cropped = segmented[min_y:max_y, min_x:max_x]
        if do_show:
            plt.imshow(cropped),plt.colorbar(),plt.show()
        return cropped

# img = extract_foreground('./CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg')

In [ ]:
# stopping point.
# next: write code to extract pre-defined parts from images
# the parts should be bounded by maximum energy.
# In order to train CNN to classify parts. There are 135 distinct labels.

# possible points of comparison: using Andrew Krause's work with randomly
# sampling POI in image.
# OR using HOG descriptor and dimensionality reduction on unsupervised method
# of finding distinct points.

# show that there is room to improve
# can we bound an improvement accuracy